In [1]:
import argparse
import torch
import numpy as np
from tqdm import tqdm
import os
from models import mlp
from data.dataset import data_loader


def test_vit(model, dataloader_test):
    """
    This function used to test ViT. 

    Args: 
        model: ViT model
        dataaloader_test: loader for test images 
    return: 
        Avg test accuracy of ViT
    
    """
    test_acc = 0.0
    for images, labels in tqdm(dataloader_test): 
        images = images.cuda()
        labels= labels.cuda()
        with torch.no_grad(): 
            model.eval()
            output = model(images)
            prediction = torch.argmax(output, dim=-1)
            acc = sum(prediction == labels).float().item()/len(labels)
            test_acc += acc
    print(f'Testing accuracy = {(test_acc/len(dataloader_test)):.4f}')

    return round(test_acc/len(dataloader_test),2)


def test_classifiers(classifiers_list, dataloader_test, mlp_root_dir, vit_model):
    for clf in range(1, len(classifiers_list) +1):
        acc_avg = 0.0
        print(classifiers_list[clf-1])
        clf_in = torch.load(os.path.join(mlp_root_dir, classifiers_list[clf-1])).cuda()
        clf_in.eval()
        # print(clf_in)
        print(f'Classifier of index {clf-1} has been loaded')

        for images, labels in tqdm(dataloader_test): 
            images = images.cuda()
            labels= labels.cuda()
            # print(images.shape) #torch.Size([30, 3, 224, 224])
            x = vit_model.patch_embed(images)
            # print(x.shape)  #torch.Size([30, 196, 768])
            x = vit_model.pos_drop(x)
            # print(x.shape)  #torch.Size([30, 196, 768])
            for block in range(clf):
                x = vit_model.blocks[block](x)
            # x = x.reshape(30, 3, 224, 224)
            with torch.no_grad():
                # print(x.shape) #torch.Size([30, 196, 768])
                output = clf_in(x)
            predictions = torch.argmax(output, dim=-1)
            acc = torch.sum(predictions == labels).item()/len(labels)
            acc_avg += acc
        print(f'Accuracy of block {clf-1} = {(acc_avg/len(dataloader_test)):.3f}')
    pass

In [5]:
root_dir = "./data/TB_data"
loader_, dataset_ = data_loader(root_dir=root_dir)
vit_block = torch.load('models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth')
vit_block.eval()
# print(type(vit_block), " - VIT_base_patch16_224")
# test_vit(model=vit_block, dataloader_test=loader_['test'])

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn

: 

In [3]:
import os
classifiers_list = sorted(os.listdir('models/MLP_new_chest'))
# print(classifiers_list)
test_classifiers(classifiers_list=classifiers_list, dataloader_test=loader_['test'], mlp_root_dir='models/MLP_new_chest', vit_model=vit_block)

block_0_classifier_0.94test_0.98train.pth
Classifier of index 0 has been loaded


100%|██████████| 23/23 [00:01<00:00, 12.26it/s]


Accuracy of block 0 = 0.945
block_1_classifier_0.93test_0.99train.pth
Classifier of index 1 has been loaded


100%|██████████| 23/23 [00:02<00:00, 10.77it/s]


Accuracy of block 1 = 0.933
block_2_classifier_0.94test_0.99train.pth
Classifier of index 2 has been loaded


...

100%|██████████| 23/23 [00:02<00:00,  8.97it/s]


Accuracy of block 3 = 0.933
block_4_classifier_0.92test_1.00train.pth
Classifier of index 4 has been loaded


100%|██████████| 23/23 [00:02<00:00,  8.61it/s]

Accuracy of block 4 = 0.923


: 

In [3]:
classifiers_list = sorted(os.listdir('models/MyModels2'))
print(classifiers_list)
test_classifiers(classifiers_list=classifiers_list, dataloader_test=loader_['test'], mlp_root_dir='models/MyModels2', vit_model=vit_block)

['model_mlp0 copy 2.pth', 'model_mlp0 copy 3.pth', 'model_mlp0 copy 4.pth', 'model_mlp0 copy.pth', 'model_mlp0.pth']


NameError: name 'vit_block' is not defined

In [4]:
classifiers_list = sorted(os.listdir('models/MyModels2'))
print(classifiers_list)
test_classifiers(classifiers_list=classifiers_list, dataloader_test=loader_['test'], mlp_root_dir='models/MyModels2', vit_model=vit_block)

['model_mlp0.pth']
model_mlp0.pth
Classifier of index 0 has been loaded


100%|██████████| 23/23 [00:01<00:00, 13.30it/s]

Accuracy of block 0 = 0.497
